In [1]:
import os
import numpy as np


import sys
sys.path.append(r"/mnt/data/GalfitX/galfitx")    ## use the path to galfitx, on 206, use '/mnt/data/GalfitX'
import galfitx

import galfitx.source_detection as se 

from galfitx.source_detection import SExtractor_HDR
from galfitx.source_detection import SExtractor_dualmode
from galfitx.source_detection import SExtractor
from galfitx.source_detection import sexcomb
from galfitx.source_detection import cat2tab
from astropy.io import fits, ascii


from photutils.background.core import SExtractorBackground, StdBackgroundRMS
from photutils.segmentation import SegmentationImage
from galfitx.postage_stamp2 import create_stamp_file, cut_stamps

import shutil

from galfitx.read_setup import read_setup, set_trailing_slash
from galfitx.sky_calculation import getsky, create_skymap
from galfitx.utils import scaled_kron
from astropy.coordinates import SkyCoord
from astropy import units as u

from galfitx import create_setup_gs


In [45]:

##### Run source detections ############
##### The detection sci image and weight image

detsci = "/home/galfitt/wangshuochun/CEERS_32/point7_F115W_bkgsub.fits"
detwht = "/home/galfitt/wangshuochun/CEERS_32/point7_F115W_wht.fits"
kernel=np.loadtxt('/mnt/data/GalfitX/galfitx/se_kernels/gauss_4.0_7x7.conv',skiprows=1)


#### This will create a dir named 'sex', and cold+hot detections.
segm_hdr, cat_hdr = SExtractor_HDR(detsci,
                                    catalog_name=('coldcat','hotcat','outcat'),
                                    segmap_name=('coldseg.fits', 'hotseg.fits', 'outseg.fits'),
                                    detect_minarea=(15, 8), detect_thresh=(3, 1.8),
                                    mask=None, coverage_mask=None,
                                    deblend_nthresh=(64, 64), 
                                    deblend=(True, True), deblend_mincont=(0.001, 0.001),
                                    clean=(True, True), clean_param=(1., 1.),
                                    back_type=(False, False), back_size=(128, 32), back_filtersize=(3, 3), bkg_estimator=SExtractorBackground(),
                                    verbose=True,
                                    weight_name=detwht,
                                    phot_autoparams=[2.5, 3.5],
                                    scale_factor=1.0,
                                    pixel_scale=0.01999999999999998,    ### change this to your pixel for detsci
                                    kernel=(kernel,kernel),
                                    mag_zeropoint=28.96697568756239,  ### change thit to your zp of sci image.
                                    weight_type='MAP_WEIGHT')

**********Cold Detection**********

Source Extraction Begins.......
detection image: /home/galfitt/wangshuochun/CEERS_32/point7_F115W_bkgsub.fits


mean weight scaling factor= 0.560659
Weight Image Preprocessing Finished
  weight_type = MAP_WEIGHT


Manual Background Subtraction Finished
  Background value = 0.0


Detection Finished
  detect_minarea = 15
  detect_thresh = 3
  threshold (above background) = 0.020777538418769836
Found 4071 sources.




Deblending: 100%|██████████| 4071/4071 [00:47<00:00, 85.57it/s] 


Deblending Finished
  deblend_nthresh = 64
  deblend_mincont = 0.001
  deblend_connectivity = 8
  deblend_mode = exponential
  nproc = 1
Found 4908 sources after deblending.




Cleaning: 100%|██████████| 4908/4908 [00:43<00:00, 112.96it/s]


Cleaning Finished
  clean_param = 1.0
Removed 523 spurious sources by cleaning.


There are 4385 objects in the catalog.


Source Extraction Begins.......
detection image: /home/galfitt/wangshuochun/CEERS_32/point7_F115W_bkgsub.fits


mean weight scaling factor= 0.57375896
Weight Image Preprocessing Finished
  weight_type = MAP_WEIGHT


Manual Background Subtraction Finished
  Background value = 0.0


Detection Finished
  detect_minarea = 8
  detect_thresh = 1.8
  threshold (above background) = 0.012757806107401848
Found 7597 sources.




Deblending: 100%|██████████| 7597/7597 [01:21<00:00, 92.75it/s] 


Deblending Finished
  deblend_nthresh = 64
  deblend_mincont = 0.001
  deblend_connectivity = 8
  deblend_mode = exponential
  nproc = 1
Found 9123 sources after deblending.




Cleaning: 100%|██████████| 9123/9123 [02:13<00:00, 68.30it/s] 


Cleaning Finished
  clean_param = 1.0
Removed 1573 spurious sources by cleaning.


There are 7550 objects in the catalog.

**********HDR Combination**********
The cold, hot and combined region files has been created
**********HDR SExtractor Finished**********
Found 7125 sources after HDR SExtractor.


In [ ]:
bands = [115, 150, 200, 277, 356, 410, 444] 

## Reference filter and sci image, pick any one from the file name list
filename= "/home/galfitt/wangshuochun/CEERS_32/point7_F277W_bkgsub.fits"

## Sci image list
filename_list = ["/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F115W/resample/bkgsub.fits",\
                  "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F150W/resample/bkgsub.fits",\
                  "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F200W/resample/bkgsub.fits",\
                "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F277W/resample/bkgsub.fits",\
                "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F356W/resample/bkgsub.fits",\
                "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F410M/resample/bkgsub.fits",\
                "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F444W/resample/bkgsub.fits"
            ]

## Claim the dir name to store cutout images.     
outdir_list = ['/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F115W/', '/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F150W/', '/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F200W/',\
                '/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F277W/', '/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F356W/',\
                '/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F410M/','/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/F444W/']
    
## label list of all filters    
label_list = np.array(['115W','150W','200W','277W','356W','410M','444W'])

### wavelength list of all filters.
wave_list =np.array(['11500', '15000', '20000', '27700', '35600', '41000', '44400'])


### number of filters used in total.
nband= len(filename_list)

## zeropoint list of all filters.
zero_list = np.array([28.96697568756239 ,
                    28.96697568756239 ,
                    28.96697568756239 ,
                    27.461825709242483 ,
                    27.461825709242483 ,
                    27.461825709242483 ,
                    27.461825709242483 ])

### generate 'stamp' file which is used to store stamp info.
catalog_name = './sex/outcat'
outtab = ascii.read(catalog_name)
create_stamp_file(filename, catalog_name, sizefac=2.5, outfile='stamps')

In [ ]:

#### pixel scale list of input images.
pixel_l =  np.array([0.01999999999999998 ,
                0.01999999999999998 ,
                0.01999999999999998 ,
                0.03999999999999996 ,
                0.03999999999999996 ,
                0.03999999999999996 ,
                0.03999999999999996 ,])


## cut image stamps
for i in range(nband):
    ps = pixel_l[i]
    filename = filename_list[i]
    outdir = outdir_list[i]
    if os.path.exists(outdir):
        shutil.rmtree(outdir)
    os.mkdir(outdir)
    cut_stamps(filename, outdir, stampfile='stamps',ps=ps,index=1)
    


In [6]:
detsci = "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F115W/resample/bkgsub.fits"
detwht = "/home/galfitt/wangshuochun/CEERS_32/point7_F115W_wht.fits"
kernel=np.loadtxt('/mnt/data/GalfitX/galfitx/se_kernels/gauss_4.0_7x7.conv',skiprows=1)

### cal sky & scale kron aperture?
skymap=create_skymap(detwht, catalog_name, 'skymap.fits', 1.1, 4)
print('create_skymap done')
scaled_kron(outtab, scale=1.1, offset=4, pixel_scale=0.03)
print('scaled_kron done')

create_skymap done
scaled_kron done


In [ ]:
# sorting mag from birght to faint
sorted_idx = np.argsort(outtab['mag_auto'])


# define fitting output directory
# path='/home/galfitt/wangshuochun/CEERS_32/GalfitX/point7/fit_output'
path='./galfits/'

### filter name of Galfits.
filter_list = np.array(['nircam_f277w','nircam_115W','nircam_150W','nircam_200W','nircam_277W','nircam_356W','nircam_410M','nircam_444W'])

### Claim the sky-value list as initial guess for galfits fitting.
cstsky_list = np.zeros(nband)


## exposure time list to calculate noise maps.
exptlist = np.array([10028.14 ,
                    2834.508 ,
                    2834.508 ,
                    2834.508 ,
                    4702.704000000001 ,
                    2834.508 ,
                    2834.508 ])


abspath = '/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Ready_Data/Pointing_7/PSF'
#### psf file list
psf_list =np.array([abspath+'F115W_hybrid.fits',\
           abspath+ 'F150W_hybrid.fits',\
           abspath+ 'F200W_hybrid.fits',\
           abspath+ 'F277W_hybrid.fits',\
           abspath+ 'F356W_hybrid.fits',\
           abspath+ 'F410M_psf.fits',\
           abspath+ 'F444W_psf.fits'
          ])

from reproject import reproject_interp

def reproject_segm(segname, sciname,output=None):
    hduseg = fits.open(segname)[0]
    hdusci = fits.open(sciname)[1]
    outseg, _ = reproject_interp(
        hduseg,
        hdusci.header,
        order='nearest-neighbor'
    )
    
    outseg[hdusci.data == 0] = 0
    
    if output is not None:
        fits.writeto(
            output,
            outseg,
            hdusci.header,
            overwrite=True
        )
    else:
        fits.writeto(
            sciname.replace('.fits','_seg.fits'),
            outseg,
            hdusci.header,
            overwrite=True
        )
        
    return 1

## create segm images if the input image has different pixel scales from your reference image.    
## For smacs, they are 4 MIRI filters.
## The input are seg images (has the same pixel scale of ref band) and sci image.
reproject_segm('./sex/outseg.fits',"/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F115W/resample/bkgsub.fits", output="/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F115W.fits")
reproject_segm('./sex/outseg.fits',"/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F150W/resample/bkgsub.fits", output="/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F150W.fits")
reproject_segm('./sex/outseg.fits',"/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F200W/resample/bkgsub.fits", output="/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F200W.fits")

## update the paths to your seg image list.
seglist = np.array(['/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F115W.fits',\
            '/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F150W.fits',\
            '/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F200W.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits'])

use_sigmap=False


In [7]:
## update the paths to your seg image list.
seglist = np.array(['/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F115W.fits',\
            '/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F150W.fits',\
            '/home/galfitt/wangshuochun/CEERS_32/GalfitX/updated_segm_F200W.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits',\
            './sex/outseg.fits'])

filename_list = ['/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F115W_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F150W_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F200W_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F277W_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F356W_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F410M_bkgsub.fits',
                    '/home/galfitt/wangshuochun/CEERS_32/zhongyi_data/Pointing_7_F444W_bkgsub.fits', ]


In [8]:
    
from tqdm import tqdm
for idx in tqdm(sorted_idx):
           
    #### could be anything, not used in the function.      
    skyfile = np.array([path+'skyfile'+label_list[j]+str(idx+1) for j in range(nband)])
    
    
    image_list = np.array([abspath+outdir_list[j]+str(idx+1)+'.fits' for j in range(nband)])
    for nn in range(nband):        
        # print(filename_list[nn])
        # print(abspath+outdir_list[nn]+str(idx+1)+'.fits')
        # print(label_list[nn])
         ### create mask images, saved in dir './galfits'
        corner, objects = create_setup_gs.create_mask(filename_list[nn], \
                                      seglist[nn], \
                                        catalog_name, \
                                      'stamps', \
                                        path+'mask'+str(idx+1), \
                                        path+'mask_primary'+str(idx+1), \
                                        current= idx,\
                                        image_stamp_list= None,\
                                        label_list=label_list[nn],\
                                        outpath = './galfits/',\
                                        ps = pixel_l[nn]
                                        )

    # image_stamp_list=  abspath+outdir_list[nn]+str(idx+1)+'.fits'
    
    
    
    ### use_sigma =False, doesn't matter here.
    if use_sigmap:
        sigma_stamp_list = np.array([outdir_list[j]+str(idx+1)+'sigma.fits' for j in range(nband)])
    else:
        sigma_stamp_list = np.array(['none']*nband)

    
    
    ### claim input images for prepare_galfits function.
    image_stamp_list = np.array([abspath+'galfits/'+str(idx+1)+'_'+label_list[j]+'sci.fits' for j in range(nband)])
    sig_stamp_list = image_stamp_list
    mask_list = np.array([abspath+'galfits/mask'+str(idx+1)+'_'+label_list[j]+'.fits' for j in range(nband)])

    ### create sigma images.
    ### You might need to check the function and add gain info if your filters are not in the list.
    create_setup_gs.create_sigma(image_list,filter_list=filter_list,label_list=label_list ,expt_list=exptlist,outpath = './galfits/')
    sig_stamp_list = np.array([abspath+'galfits/'+str(idx+1)+'_'+label_list[j]+'sigma.fits' for j in range(nband)])
    
    
    
    ### pause here.
    # ### creating galfits configuration file.
    # create_setup_gs.prepare_galfits(image_stamp_list,\
    #                 catalog_name, \
    #                 path+'obj'+str(idx+1),corner,\
    #                 [path+'mask'+str(idx+1)]*nband,\
    #                 'stamps', psf_list, skyfile, zero_list, label_list, filter_list,mask_file_list=mask_list,\
    #                 nband=nband, objects=objects, Ncomps_primary=1, sig_stamp_list=sig_stamp_list, setup=setup,sky0=True,max_r=5)
    

       

  0%|          | 0/7125 [00:00<?, ?it/s]

115W 15349 5617 6052 4225


ValueError: negative dimensions are not allowed

In [ ]:
 ### create mask images, saved in dir './galfits'
        corner, objects = create_setup_gs.create_mask(filename_list[nn], \
                                      seglist[nn], \
                                        catalog_name, \
                                      'stamps', \
                                        path+'mask'+str(idx+1), \
                                        path+'mask_primary'+str(idx+1), \
                                        current= idx,\
                                        image_stamp_list= None,\
                                        label_list=label_list[nn],\
                                        outpath = './galfits/',\
                                        ps = pixel_l[nn]
                                        )

# image_stamp_list=  abspath+outdir_list[nn]+str(idx+1)+'.fits'
    ### use_sigma =False, doesn't matter here.
    if use_sigmap:
        sigma_stamp_list = np.array([outdir_list[j]+str(idx+1)+'sigma.fits' for j in range(nband)])
    else:
        sigma_stamp_list = np.array(['none']*nband)
    
    
    
    ### claim input images for prepare_galfits function.
    image_stamp_list = np.array([abspath+'galfits/'+str(idx+1)+'_'+label_list[j]+'sci.fits' for j in range(nband)])
    sig_stamp_list = image_stamp_list
    mask_list = np.array([abspath+'galfits/mask'+str(idx+1)+'_'+label_list[j]+'.fits' for j in range(nband)])

    ### create sigma images.
    ### You might need to check the function and add gain info if your filters are not in the list.
    create_setup_gs.create_sigma(image_list,filter_list=filter_list,label_list=label_list ,expt_list=exptlist,outpath = './galfits/')
    sig_stamp_list = np.array([abspath+'galfits/'+str(idx+1)+'_'+label_list[j]+'sigma.fits' for j in range(nband)])
    
    
    
    ### pause here.
    # ### creating galfits configuration file.
    # create_setup_gs.prepare_galfits(image_stamp_list,\
    #                 catalog_name, \
    #                 path+'obj'+str(idx+1),corner,\
    #                 [path+'mask'+str(idx+1)]*nband,\
    #                 'stamps', psf_list, skyfile, zero_list, label_list, filter_list,mask_file_list=mask_list,\
    #                 nband=nband, objects=objects, Ncomps_primary=1, sig_stamp_list=sig_stamp_list, setup=setup,sky0=True,max_r=5)
    


In [24]:

from astropy.wcs import WCS

bands = [115, 150, 200, 277, 356, 410, 444]  #setect band
# bands = [277]
for b in bands:
    suffix = 'M' if b == 410 else 'W'
    band_name = f"F{b}{suffix}"

    # fname = f"/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F115W/resample/nircam_F115W_mosaic_resample.fits"
    fname = f"/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/{band_name}/resample/bkgsub.fits"
    with fits.open(fname) as hdul:

        data_full = hdul[1].data 
        header = hdul[1].header
        wcs_full = WCS(header)
        # print(repr(header))
        print(header["XPOSURE"],",")
        # print(hdul.info)
   
        # new_hdu = fits.PrimaryHDU(data=data_full, header=header)
        # new_hdu.writeto("/home/galfitt/wangshuochun/CEERS_32/point7_F115W_wht.fits", overwrite=True)

# print(data_full.shape)
        # dist_scale = d.to(u.kpc).value*np.pi/180/3600 # kpc per arcsec

        # pixel_scale =  wcs_full.proj_plane_pixel_scales()[0].to(u.arcsec).value  
        # scale = pixel_scale*dist_scale   # kpc/pixel

        # pixar_sr = header["PIXAR_SR"]
        # ZP=-6.10-2.5*np.log10(pixar_sr)
        # print(ZP,",")

10028.14 ,
2834.508 ,
2834.508 ,
2834.508 ,
4702.704000000001 ,
2834.508 ,
2834.508 ,


In [44]:
fname = "/RS2423/JWST/BLAGN_legacy/CEERS_PID_1345_NIRCam/Pointing_7/F115W/resample/bkgsub.fits"

with fits.open(fname) as hdul:

    data_full = hdul[1].data 
    header = hdul[1].header
    wcs_full = WCS(header)

new_hdu = fits.PrimaryHDU(data=data_full, header=header)
new_hdu.writeto("/home/galfitt/wangshuochun/CEERS_32/point7_F115W_bkgsub.fits", overwrite=True)
